# Animation visualization of dynamic networks
## Animation layout

Here we use the following animation panel layout. Each row for c,d,e,f has their own set of select TFs. The layout is arbitrary and you can design your own custom layout with `matplotlib`.

```
+---+       +---+
| a |       | b |
+---+---+---+---+
| c | d | e | f |
+---+---+---+---+
| c | d | e | f |
+---+---+---+---+
```

* a: Dynamic tracking of cells used for GRN inference
* b: Dynamic scatter plot for differential regulation v.s. differential expression logFC
* c: Dynamic plot for expression level (log CPM) of select TFs as a function of pseudo-time
* d: Dynamic plot for regulatory activity (log target count) of select TFs as a function of pseudo-time
* e: Dynamic heatmap for regulation strength from select TFs to select target genes
* f: Dynamic subnetwork graph from select TF to its targets

## Configuring matplotlib to enable large animation

In [1]:
%config IPCompleter.use_jedi=False
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt


## Load data

In [2]:
import itertools
from functools import partial
import dictys
from dictys.net import dynamic_network
from dictys.net.layout import _fruchterman_reingold
from dictys.net import stat
from dictys.plot import panel

d0=dynamic_network.from_file('../../data/dynamic.h5')


## Choosing branch to draw and genes to annotate

In [3]:
#Determine branch with (starting node, ending node) from trajectory inference
#See example trajectory-blood
#Monocyte branch
# branch=(0,1)
#Erythroid branch
branch=(0,2)
#B cell branch
# branch=(0,3)
#Select TFs for each row's dynamic subnetwork graph
tfs_subnet=[
	['HLF'],
	['GATA1'],
]
#Select TFs for each row's other plots
tfs_ann=[
	['MYCN', 'HLF'],
	['GATA1', 'KLF1', 'HLTF', 'TAL1'],
]
#Select genes to annotate as targets in all rows
target_ann=['CD34', 'GYPC', 'PRIM1', 'TRIM58', 'XPO7', 'YOD1', 'BTRC','FBXO9', 'PRPS1', 'MTMR3', 'MTMR12','MAZ','SLC2A1','PPOX','ADD2','ALAD','CDC20','NUSAP1','E2F2']

n=len(tfs_ann)
assert len(tfs_subnet)==n


## Branch independent parameters

In [4]:
#Kernel smoothing distance
dist=1.5
#Number of frames (interpolated time points), use 100 or higher for finer visualization
nframe=20
#Animation FPS for saving. Determines speed of play
fps=0.10*nframe
#Size of each panel
panelsize=(6,4)
#DPI for animation
dpi=200
#Bit rate for each panel, each FPS, and each dot per square inch. Increase for higher vidoe quality. Decrease for smaller file size.
bitrate=1/400
#bitrate*len(axes)*fps*dpi**2)
#Number of panels per row/column
panelcount=(4,1+n)


## Kernel smoothed network properties


In [5]:
pts,fsmooth=d0.linspace(branch[0],branch[1],nframe,dist)
# Expression: logCPM
stat1_lcpm=fsmooth(stat.lcpm(d0,cut=0))
# Kernel smoothed network
stat1_net=fsmooth(stat.net(d0))
# Binarized network
stat1_netbin=stat.fbinarize(stat1_net)
# You can change network sparsity with: stat1_netbin=stat.fbinarize(stat1_net,sparsity=0.001)
# Regulatory activity: log target count
stat1_lntarget=stat.flnneighbor(stat1_netbin)
# Pseudo time
stat1_pseudotime=stat.pseudotime(d0,pts)


## Kernel smoothed network properties for each row or panel

In [6]:
tf_ann=list(set(itertools.chain.from_iterable(tfs_ann)))
#Selecting TF's outgoing edges as subnetwork
stat1_subnets=[stat1_net[x] for x in tfs_subnet]
stat1_subnetbins=[stat1_netbin[x] for x in tfs_subnet]
stat1_subnet_truncs=[stat.function(lambda *y:y[0]*y[1],x,names=x[0].names) for x in zip(stat1_subnets,stat1_subnetbins)]
#Performing layout with linear smoothing of node locations
weightfunc_linear=['linear',[],dict()]
stat1_layouts=[stat.fsmooth(stat.flayout_base(x,partial(_fruchterman_reingold,stop=20,iterations=50),pts=pts),pts,weightfunc_linear) for x in stat1_subnet_truncs]


## Draw animation

In [ ]:
#Animation formating
fig=plt.figure(figsize=(panelsize[0]*panelcount[0],panelsize[1]*panelcount[1]),dpi=dpi)
axes=[fig.add_subplot(*panelcount[::-1],x+1) for x in range(panelcount[0]*panelcount[1])]
[[y.set_visible(False) for y in x.spines.values()] for x in axes[:3]]
for ax in axes[3:15]:
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
plt.subplots_adjust(wspace=0.35,hspace=0.2)

#Drawing each panel with iterator
panels=[]
axes_iter=iter(axes)

#Panel a
panels.append(panel.cellscatter(next(axes_iter),d0,pts,fsmooth))
#Create two empty panels to skip
next(axes_iter)
next(axes_iter)
#Panel b
panels.append(panel.statscatter(next(axes_iter),pts,stat.fdiff(stat1_lcpm,stat.finitial(stat1_lcpm,pts),label='Diff express (LogFC in CPM)'),stat.fdiff(stat1_lntarget,stat.finitial(stat1_lntarget,pts),label='Diff regul (LogFC in target count)'),annotate=tf_ann,aspect=1,lim={'sym','min','max'}))
#Each row
for xi in range(n):
	#Panel c
	panels.append(panel.statplot(next(axes_iter),pts,stat1_pseudotime,stat1_lcpm,names=tfs_ann[xi]))
	#Panel d
	panels.append(panel.statplot(next(axes_iter),pts,stat1_pseudotime,stat1_lntarget,names=tfs_ann[xi]))
	#Panel e
	panels.append(panel.statheatmap(next(axes_iter),pts,stat1_net,names=[tfs_ann[xi],target_ann],lim=[-0.3,0.3],cmap='coolwarm',aspect='auto'))
	#Panel f
	panels.append(panel.network(next(axes_iter),pts,stat1_layouts[xi],stat1_subnet_truncs[xi],nodeka={'annotate':tfs_subnet[xi],'scatterka':{'s':5,'lw':0}},edgeka={'lw':0.05}))

#Draw animation
ca=panel.animate_generic(pts,fig,panels)
anim=ca.animate()

#Saving animation methods (choose one):
#1. Visualize animation within jupyter. This will make the notebook large especially with many frames
# from IPython.display import HTML
# HTML(anim.to_jshtml())
#2. Save animation to mp4 file
w=matplotlib.animation.writers['ffmpeg_file'](fps=fps,bitrate=bitrate*len(axes)*fps*dpi**2)
w.frame_format='jpeg'
anim.save(f'../output/animation-branch{branch[1]}.mp4',writer=w,dpi='figure')
